In [ ]:
import tensorflow as tf ## pip install tensorflow
import cv2 ## pip install opencv-contrib-python
import numpy as np ## pip install numpy
import matplotlib.pyplot as plt  ## pip install matplotlib
import os
!set KMP_DUPLICATE_LIB_OK=TRUE

In [ ]:
Datadirectory = "Training" ##training dataset


In [ ]:
Classes = ["0","1","2","3","4","5","6"] ##List of classes=> exact name of your folders

## Testing By Printing a single image from folder '0'

In [ ]:
img_array = cv2.imread("Training/0/Training_3908.jpg")

In [ ]:
img_array.shape #rgb

In [ ]:
plt.imshow(img_array)

## Preprocessing Demo on a single image

In [ ]:
for category in Classes:
    path = os.path.join(Datadirectory,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        #backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
        plt.imshow(cv2.cvtColor(img_array,cv2.COLOR_BGR2RGB))
        plt.show()
        break
    break

In [ ]:
img_size=224 ##ImageNet => 224x224
new_array=cv2.resize(img_array, (img_size,img_size))
plt.imshow(cv2.cvtColor(new_array,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
new_array.shape
## Now the image size is changed as needed for the input into ImageNet CNN Architecture that we are gonna use

 # Read all images and convert them to array

In [ ]:
training_Data=[] ##data

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory,category)
        class_num = Classes.index(category)  ## 0 1,  ## Label
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (img_size,img_size))
                training_Data.append( [new_array,class_num] )
            except Exception as e:
                pass
               
        
    

In [ ]:
create_training_Data()
print(len(training_Data))

In [ ]:
import random ## shuffling the images into random order
random.shuffle(training_Data)

In [ ]:
x = [] ## data/feature
y = [] ## label
for features,label in training_Data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, img_size, img_size, 3) ## converting it to 4 dimensions

In [ ]:
x.shape  ## Changing the dimension because the deep learning architecture such as mobileNet requires 4 dimensions

In [ ]:
# Normalizing the data
X = x/255.0
X.shape

In [ ]:
x.shape

In [ ]:
Y = np.array(y)
Y.shape

In [ ]:
from collections import Counter


# Count the occurrences of each value
counts = Counter(Y)

# Print the counts
for value, count in counts.items():
    print(f"Value {value}: {count}")


# deep Learning model for training - Transfer Learning

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
model = tf.keras.applications.MobileNetV2() ## Pre-trained Model

In [ ]:
model.summary()

# Transfer Learning - Tuning, weights will start from last check point

In [ ]:
base_input = model.layers[0].input

In [ ]:
base_input

In [ ]:
base_ouput = model.layers[-2].output

In [ ]:
base_ouput

In [ ]:
final_output = layers.Dense(128)(base_ouput) ## adding new layer after output of global pooling layer
final_ouput  = layers.Activation('relu')(final_output) ## activation function
final_output = layers.Dense(64)(final_ouput)
final_ouput  = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_ouput) ## we need 7 classes ## Classification layer

In [ ]:
final_output

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Model

new_model = keras.Model(inputs = base_input, outputs = final_output)
new_model.summary()

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer= "adam", metrics = ["accuracy"] )

In [ ]:
new_model.fit(X, Y, epochs=30)
new_model.save('fer_model_2.h5')
## Training is completed and model saved as a file names 'fer_model_2.h5' in base directory

## Testing the model

In [ ]:
new_model = tf.keras.models.load_model('fer_model.h5')  ## Load the model saved after training from the previous step

In [ ]:
frame = cv2.imread("happyboy.jpg")
frame.shape

## we need face detection algorithm (gray image)

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

In [ ]:
gray.shape

In [ ]:
faces = faceCascade.detectMultiScale(gray,1.1,4)
for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:
        print("Face not detected")
    else:
        for (ex,ey,ew,eh) in facess:
            face_roi = roi_color[ey: ey+eh, ex:ex+ew] ## Cropping the face

In [ ]:
plt.imshow( cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow( cv2.cvtColor(face_roi,cv2.COLOR_BGR2RGB))

In [ ]:
final_image = cv2.resize(face_roi,(224,224)) ## Converting to 224x224
final_image = np.expand_dims(final_image,axis=0) ## need fourth dimension
final_image = final_image/255.0 ## normalizing

In [ ]:
Predictions = new_model.predict(final_image)
Predictions[0]

In [ ]:
np.argmax(Predictions)